In [166]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import json
import re
from bs4 import BeautifulSoup
import requests
%matplotlib inline

In [8]:
# Test dataframe with only yummly data

# cuisines = []
# flavors = []
# ingredients = []
# name = []
# source = []
# for filename in os.listdir('data/metadata27638'):
#     if filename.endswith('.json'):
#         data = json.load(open('data/metadata27638/'+filename))
#         cuisines.append(data['attributes']['cuisine'][0])
#         flavors.append(data['flavors'])
#         ingredientStr = ''
#         for ingred in data['ingredientLines']:
#             ingredientStr = ingredientStr + ingred
#         ingredients.append(ingredientStr)
#         name.append(data['name'])
#         source.append(data['source']['sourceDisplayName'])
        
# df = pd.DataFrame({'cuisine':cuisines,'name':name,'flavors':flavors,'ingredients':ingredients,'source':source})

In [41]:
# Reading in data and checking if name and ingredient lists exist

names = []
ingreds = []

for filename in os.listdir('data/recipe_box'):
    if filename.endswith('.json'):
        data = list(json.load(open('data/recipe_box/'+filename)).values())
        for recipe in data:
            if ('title' not in recipe.keys() or 'ingredients' not in recipe.keys()):
                continue
            names.append(recipe['title'])
            ingreds.append(recipe['ingredients'])

In [42]:
# Creating data frame with dish names and ingredients

df = pd.DataFrame({'name':names,'ingredients':ingreds})

In [45]:
df.head()

,name,ingredients
0,Grammie Hamblet's Deviled Crab,"[1/2 cup celery, finely chopped, 1 small green..."
1,Infineon Raceway Baked Beans,"[2 pounds skirt steak, cut into 1/2-inch dice,..."
2,Southwestern Black Bean Dip,"[1 1/2 cups dried black beans, picked over and..."
3,Sour Cream Noodle Bake,"[1 1/4 pounds ground chuck, One 15-ounce can t..."
4,Sushi Renovation,"[1 cup rice, brown, medium-grain, cooked, 1/2-..."


In [47]:
# Example of ingredient list

df['ingredients'][10]

['3 to 4 cups chicken stock, preferably homemade, recipe follows',
 '1 quart Roasted Winter Vegetables, recipe follows',
 'Kosher salt and freshly ground black pepper',
 '3 (5-pound) roasting chickens',
 '3 large yellow onions, unpeeled, quartered',
 '6 carrots, unpeeled, halved',
 '4 celery stalks with leaves, cut in thirds',
 '4 parsnips, unpeeled, cut in 1/2, optional',
 '20 sprigs fresh parsley',
 '15 sprigs fresh thyme',
 '20 sprigs fresh dill',
 '1 head garlic, unpeeled, cut in 1/2 crosswise',
 '2 tablespoons kosher salt',
 '2 teaspoons whole black peppercorns',
 '1 pound carrots, peeled',
 '1 pound parsnips, peeled',
 '1 large sweet potato, peeled',
 '1 small butternut squash (about 2 pounds), peeled and seeded',
 '3 tablespoons good olive oil',
 '1 1/2 teaspoons kosher salt',
 '1/2 teaspoon freshly ground black pepper',
 '2 tablespoons chopped fresh flat-leaf parsley']

In [48]:
# Example of ingredient list with ADVERTISEMENT tags

df['ingredients'][100000]

['1/2 small head green cabbage, cored and thinly sliced ADVERTISEMENT',
 '1/2 jicama, sliced into matchsticks ADVERTISEMENT',
 '1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT',
 '1/2 cup mayonnaise ADVERTISEMENT',
 '1/4 cup pineapple juice ADVERTISEMENT',
 '1 teaspoon white sugar ADVERTISEMENT',
 'hot sauce to taste ADVERTISEMENT',
 'salt and freshly ground black pepper to taste ADVERTISEMENT',
 '1/4 bunch chopped fresh cilantro ADVERTISEMENT',
 '1/3 ounce toasted corn bits (such as CornNuts ®), crushed ADVERTISEMENT',
 'ADVERTISEMENT']

In [122]:
testlist = df['ingredients'][100000]
testlist

['1/2 small head green cabbage, cored and thinly sliced ADVERTISEMENT',
 '1/2 jicama, sliced into matchsticks ADVERTISEMENT',
 '1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT',
 '1/2 cup mayonnaise ADVERTISEMENT',
 '1/4 cup pineapple juice ADVERTISEMENT',
 '1 teaspoon white sugar ADVERTISEMENT',
 'hot sauce to taste ADVERTISEMENT',
 'salt and freshly ground black pepper to taste ADVERTISEMENT',
 '1/4 bunch chopped fresh cilantro ADVERTISEMENT',
 '1/3 ounce toasted corn bits (such as CornNuts ®), crushed ADVERTISEMENT',
 'ADVERTISEMENT']

In [190]:
def clean_recipe(ingred_list):
    cleanedtest = []
    for ingred in ingred_list:
        # Ignore extra information
        matchtext = re.sub(r' \([^)]*\)', '', ingred)
        
        # Obtain all before first comma
        if re.compile('^(.+?)(?=,)').search(matchtext) is not None:
            matchtext = re.compile('^(.+?)(?=,)').search(matchtext).group(1)
            matchtext = matchtext.replace(' ADVERTISEMENT','')
            cleanedtest.append(matchtext)
            
        # Ignore advertisement tag
        
        elif (ingred == 'ADVERTISEMENT'):
            continue
        
        # If all text, make sure no advertisement tag
        else:
            matchtext = matchtext.replace(' ADVERTISEMENT','')
            cleanedtest.append(matchtext)
    return cleanedtest

In [191]:
clean_recipe(df['ingredients'][100000])

['1/2 small head green cabbage',
 '1/2 jicama',
 '1 large sweet apple',
 '1/2 cup mayonnaise',
 '1/4 cup pineapple juice',
 '1 teaspoon white sugar',
 'hot sauce to taste',
 'salt and freshly ground black pepper to taste',
 '1/4 bunch chopped fresh cilantro',
 '1/3 ounce toasted corn bits']

In [155]:
#regex test
# re.compile('(?<=\d )(?!.*\d)(.+?)(?=[^a-zA-Z ])').search('1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT').group(1)

In [188]:
# Scraping corpus of words related to measurements

page = requests.get('https://www.enchantedlearning.com/wordlist/measurement.shtml')
soup = BeautifulSoup(page.content, "html.parser")
measure_corpus = [tag.text for tag in soup.find_all('div',attrs={'class':'wordlist-item'})]

In [189]:
measure_corpus

['acre',
 'angstrom',
 'area',
 "baker's dozen",
 'bale',
 'barrel',
 'bit',
 'bolt',
 'breadth',
 'bulk',
 'bushel',
 'byte',
 'carat',
 'case',
 'centimeter',
 'cord',
 'cord foot',
 'cubic',
 'cubic centimeter',
 'cubic inch',
 'cubic meter',
 'cubic mile',
 'cubic yard',
 'cubit',
 'cup',
 'dash',
 'decigram',
 'deciliter',
 'decimal',
 'decimeter',
 'degree',
 'depth',
 'digit',
 'dozen',
 'dram',
 'drop',
 'dry gallon',
 'dry quart',
 'em',
 'en',
 'fathom',
 'feet',
 'fermi',
 'fluid ounce',
 'folio',
 'foot',
 'furlong',
 'gallon',
 'gauge',
 'gill',
 'grain',
 'gram',
 'grit',
 'gross',
 'gross ton',
 'hand',
 'hectare',
 'height',
 'hundredweight',
 'inch',
 'jerib',
 'karat',
 'kilo',
 'kilogram',
 'kilometer',
 'knot',
 'league',
 'length',
 'light-second',
 'light-year',
 'liquid ounce',
 'liter',
 'mass',
 'measure',
 'megameter',
 'megaparsec',
 'megapixel',
 'megaton',
 'meter',
 'metric system',
 'microgram',
 'microliter',
 'micron',
 'mile',
 'milligram',
 'millilite